# Apply trace metal concentrations to river classification

In [1]:
import numpy as np
import netCDF4 as nc
import xarray as xr

%matplotlib inline

## Runoff categories:

1. Glaciers
2. Continental
3. Central

In [2]:
def create_forcing_fields(class_file, SPM_factor=0.0, multiply_glacial=1.0, multiply_continental=1.0):
    
    rclass = nc.Dataset(f'/ocean/brogalla/GEOTRACES/data/{class_file}','r')
    river_class = np.array(rclass.variables['rclass'])
    
    SPM_rivers = np.zeros(river_class.shape)
    dMn_rivers = np.zeros(river_class.shape)
    
    dMn_rivers[river_class==1.0] = 164e-9*multiply_glacial       # glaciers
    dMn_rivers[river_class==2.0] = 29.8e-9*multiply_continental  # continental
    dMn_rivers[river_class==3.0] = 2.2e-9  # other
    dMn_rivers[river_class==4.0] = 2.2e-9   
    
    SPM_rivers[river_class==1.0] = SPM_factor*261e-6*multiply_glacial        # glaciers
    SPM_rivers[river_class==2.0] = SPM_factor*1.196e-5*multiply_continental  # continental
    SPM_rivers[river_class==3.0] = SPM_factor*4.08e-6   # other
    SPM_rivers[river_class==4.0] = SPM_factor*4.08e-6  
    
    return SPM_rivers, dMn_rivers

In [3]:
def save_file(SPM_rivers, dMn_rivers, name='Mn-river-base'):
    file_write = xr.Dataset(
        {'spm_rivers': (("y","x"), SPM_rivers), 
         'dmn_rivers': (("y","x"), dMn_rivers)}, 
        coords = {
            "y": np.zeros(2400),
            "x": np.zeros(1632),
        },
    )

    file_write.to_netcdf(f'/ocean/brogalla/GEOTRACES/data/paper2-forcing-files/{name}.nc')
    
    return

### Apply trace metal concentrations

In [4]:
base_SPM       , base_dMn        = create_forcing_fields('river_class-202005.nc', SPM_factor=0)
glacial_SPM    , glacial_dMn     = create_forcing_fields('river_class-202005.nc', SPM_factor=0, multiply_glacial=1.5)
continental_SPM, continental_dMn = create_forcing_fields('river_class-202005.nc', SPM_factor=0, multiply_continental=1.5)

### Write to NetCDF file

In [5]:
save_file(base_SPM       , base_dMn       , name='river-forcing-base-202112')
save_file(glacial_SPM    , glacial_dMn    , name='river-forcing-glacial-202112')
save_file(continental_SPM, continental_dMn, name='river-forcing-continental-202112')